<a href="https://colab.research.google.com/github/seunghee0518/python_colab/blob/main/tensorflow_car_evaluation_%EC%9E%90%EB%A3%8C%EB%A1%9C_%EB%B6%84%EB%A5%98_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install& import library

In [ ]:
#library install
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

In [ ]:
#(must!) upgrade
!pip install --upgrade pandas --use-feature=2020-resolver
!pip install --upgrade matplotlib --use-feature=2020-resolver
!pip install --upgrade seaborn --use-feature=2020-resolver

In [ ]:
#tensorflow library
!pip install -q tensorflow-gpu==2.0.0-rc1

In [ ]:
#import library
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

# load data

In [ ]:
#data link : https://github.com/gilbutITbook/080263

In [ ]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety', 'output']
car_url = 'https://raw.githubusercontent.com/gilbutITbook/080263/master/chap2/data/car_evaluation.csv'
cars = pd.read_csv(car_url, names=cols, header = None)
cars

In [ ]:
#pie-chart
plot_size = plt.rcParams["figure.figsize"]
plot_size[0] = 8
plot_size[1] = 6
plt.rcParams["figure.figsize"] = plot_size
cars.output.value_counts().plot(kind='pie', autopct = '%0.05f%%', colors = ['lightblue', 'lightgreen', 'orange', 'pink'] , explode = (0.05, 0.05, 0.05, 0.05))

* output(차 상태)
  * unacc : 허용 불가능 상태
  * acc : 허용 가능한 수준
  * good : 양호
  * very good : 매우 좋은 상태
* 70%가 허용 불가능

#preprocess
* get_dummies() : 가변수 만듦, 문자열만 인코딩 됨, 숫자X
* train_test_split() : 훈련/검증 데이터 분리

In [ ]:
#one-hot encording
price = pd.get_dummies(cars.price, prefix = 'price')
maint = pd.get_dummies(cars.maint, prefix = 'maint')
doors = pd.get_dummies(cars.doors, prefix = 'doors')
persons = pd.get_dummies(cars.persons, prefix = 'persons')
lug_capacity = pd.get_dummies(cars.lug_capacity, prefix = 'lug_capacity')
safety = pd.get_dummies(cars.safety, prefix = 'safety')
labels = pd.get_dummies(cars.output, prefix = 'condition')

X = pd.concat([price, maint, doors, persons, lug_capacity, safety], axis = 1)
y = labels.values

In [ ]:
#train/test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
#create model & compile
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.models import Model

input_layer = Input(shape = (X.shape[1],))
dense_layer_1 = Dense(15, activation = 'relu')(input_layer)
dense_layer_2 = Dense(10, activation = 'relu')(dense_layer_1)
output = Dense(y.shape[1], activation = 'softmax')(dense_layer_2) #이때 마지막 밀집층은 노드 4개임 y.shape[1] == 4
model = Model(inputs = input_layer, outputs = output)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

model.summary()

In [ ]:
#training model
history = model.fit(X_train, y_train, batch_size = 8, epochs = 50, verbose=1, validation_split = 0.2)

In [ ]:
#evaluate model
score = model.evaluate(X_test, y_test, verbose = 1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])